In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf
import tensorflow_probability as tfp

tfd = tfp.distributions
tfb = tfp.bijectors

In [ ]:
def print_subclasses_from_module(module, base_class, maxwidth=80):
  import functools, inspect, sys
  subclasses = [name for name, obj in inspect.getmembers(module)
                if inspect.isclass(obj) and issubclass(obj, base_class)]
  def red(acc, x):
    if not acc or len(acc[-1]) + len(x) + 2 > maxwidth:
      acc.append(x)
    else:
      acc[-1] += ", " + x
    return acc
  print('\n'.join(functools.reduce(red, subclasses, [])))

In [ ]:
mats = tf.random.uniform(shape=[1000, 10, 10])
vecs = tf.random.uniform(shape=[1000, 10, 1])

def for_loop_solve():
  return np.array(
    [tf.linalg.solve(mats[i, ...], vecs[i, ...]) for i in range(1000)])

def vectorized_solve():
  return tf.linalg.solve(mats, vecs)

# Vectorization for the win!
%timeit for_loop_solve()
%timeit vectorized_solve()

In [ ]:
# Code can run seamlessly on a GPU, just change Colab runtime type
# in the 'Runtime' menu.
if tf.test.gpu_device_name() == '/device:GPU:0':
  print("Using a GPU")
else:
  print("Using a CPU")

In [ ]:
a = tf.constant(np.pi)
b = tf.constant(np.e)
with tf.GradientTape() as tape:
  tape.watch([a, b])
  c = .5 * (a**2 + b**2)
grads = tape.gradient(c, [a, b])
print(grads[0])
print(grads[1])

In [ ]:
print_subclasses_from_module(tfp.distributions, tfp.distributions.Distribution)

In [ ]:
normal = tfd.Normal(loc=0, scale=1)
print(normal)

In [ ]:
samples = normal.sample(1000)
sns.displot(samples)
plt.title("Samples from standard Normal")
plt.show()


In [ ]:
normal.log_prob(0.)

In [ ]:
normals = tfd.Normal([-2.5, 0., 2.5], 1.)
print("Batch shape: ", normals.batch_shape)
print("Event shape: ", normals.event_shape)

In [ ]:
samples = normals.sample(1000)
print("Sahpe of samples: ", samples.shape)

In [ ]:
# And so will this!
xs = np.linspace(-6, 6, 200)[..., np.newaxis]
lps = normals.log_prob(xs)
print("Broadcast log_prob shape:", lps.shape)

# Summarizing visually
for i in range(3):
  sns.displot(samples[:, i], kde=False)
plt.plot(np.tile(xs, 3), normals.prob(xs), c='k', alpha=.5)
plt.title("Samples from 3 Normals, and their PDF's")
plt.show()

In [ ]:
kernel = tfp.math.psd_kernels.ExponentiatedQuadratic()
xs = np.linspace(-5., 5., 2000).reshape([-1, 1])
